 ChIP-seq analysis

# peak calling using macs2
with INP file as the background

### the peak calling for one sample takes a about 10min time. 

In [4]:
module load macs

[+] Loading singularity  3.7.4  on cn0943 
[+] Loading macs  2.2.7.1 


In [21]:
macs2 callpeak --help | grep bdg

  -B, --bdg             Whether or not to save extended fragment pileup, and
                        bdgcmp and bdgpeakcall, you shouldn't use this option


In [5]:
#macs2 callpeak -t sam/STAT1_30m_IFNa.sam -c norm_INP_30m_IFNa.sam -n STAT1_30m_IFNa -g hs --bdg -q 0.05 -f SAM

# optimized for biowulf
SECONDS=0
macs2 callpeak -t sam/STAT1_6h_IFNa.bam -c sam/INP_6h_IFNa_11E6.bam -n STAT1_6h_IFNa -g hs --bdg -q 0.05 -f BAM

duration=$SECONDS
echo "$(($duration / 60)) minutes and $(($duration % 60)) seconds elapsed."

INFO  @ Thu, 17 Jun 2021 12:34:21: 
# Command line: callpeak -t sam/STAT1_6h_IFNa.bam -c sam/INP_6h_IFNa_11E6.bam -n STAT1_6h_IFNa -g hs --bdg -q 0.05 -f BAM
# ARGUMENTS LIST:
# name = STAT1_6h_IFNa
# format = BAM
# ChIP-seq file = ['sam/STAT1_6h_IFNa.bam']
# control file = ['sam/INP_6h_IFNa_11E6.bam']
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 1000 bps and 10000 bps
# Broad region calling is off
# Paired-End mode is off
 
INFO  @ Thu, 17 Jun 2021 12:34:21: #1 read tag files... 
INFO  @ Thu, 17 Jun 2021 12:34:21: #1 read treatment tags... 
INFO  @ Thu, 17 Jun 2021 12:34:36:  1000000 
INFO  @ Thu, 17 Jun 2021 12:34:50:  2000000 
INFO  @ Thu, 17 Jun 2021 12:35:04:  3

In [2]:
macs2 callpeak -t sam/STAT1_30m_IFNa.bam -c sam/INP_30m_IFNa_11E6.bam -n STAT1_30m_IFNa -g hs --bdg -q 0.05 -f BAM

bash: macs2: command not found


: 127

# download two scripts to convert file types

In [ ]:
wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64.v369/bedGraphToBigWig
wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64.v369/fetchChromSizes
# make them executable
chmod 755 bedGraphToBigWig 
chmod 755 fetchChromSizes 

### chomSizes will be used to normalize the depth?

In [9]:
 ./fetchChromSizes hg19 >hg19.chrom.sizes

INFO: trying CURL /usr/bin/curl for database hg19
url: http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.chrom.sizes


# Convert bedgraph to wiggle for downstream programs IGB and CEAS

In [ ]:
./bedGraphToBigWig peak/STAT1_6h_IFNa_treat_pileup.bdg hg19.chrom.sizes STAT1_6h_IFNa.bigwig
./bedGraphToBigWig peak/STAT1_6h_IFNa_control_lambda.bdg hg19.chrom.sizes INP_6h_IFNa.bigwig

# 
## selected fields from narrowPeak to form bed file

In [ ]:
cut -f1,2,3,4 STAT1_6h_IFNa_peaks.narrowPeak > STAT1_6h_IFNa_peaks.bed

In [11]:
wget gist.github.com/svigneau/8846527/raw/7fdf60b379245904f1aeb02427e098c06aeb670e/%2520bedgraph_to_wig.pl

--2021-06-14 22:24:20--  http://gist.github.com/svigneau/8846527/raw/7fdf60b379245904f1aeb02427e098c06aeb670e/%2520bedgraph_to_wig.pl
Resolving dtn07-e0 (dtn07-e0)... 10.1.200.243
Connecting to dtn07-e0 (dtn07-e0)|10.1.200.243|:3128... connected.
Proxy request sent, awaiting response... 301 Moved Permanently
Location: https://gist.github.com/svigneau/8846527/raw/7fdf60b379245904f1aeb02427e098c06aeb670e/%2520bedgraph_to_wig.pl [following]
--2021-06-14 22:24:20--  https://gist.github.com/svigneau/8846527/raw/7fdf60b379245904f1aeb02427e098c06aeb670e/%2520bedgraph_to_wig.pl
Connecting to dtn07-e0 (dtn07-e0)|10.1.200.243|:3128... connected.
Proxy request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/svigneau/8846527/raw/7fdf60b379245904f1aeb02427e098c06aeb670e/%2520bedgraph_to_wig.pl [following]
--2021-06-14 22:24:20--  https://gist.githubusercontent.com/svigneau/8846527/raw/7fdf60b379245904f1aeb02427e098c06aeb670e/%2520bedgraph_to_wig.pl
Conn

In [12]:
# mv %20bedgraph_to_wig.pl bedgraph_to_wig.pl

# Convert bedgraph to wig file (signal file?), for visualization in the IGB browser

In [23]:
perl bedgraph_to_wig.pl --help

Unknown option: help
Error in command line arguments!
Usage: bedgraph_to_wig.pl --bedgraph <infile.bedgraph> --wig <outfile.wig> --step <step_size> [--compact]



: 255

In [ ]:
perl bedgraph_to_wig.pl --bedgraph peak/STAT1_6h_IFNa_treat_pileup.bdg --wig STAT1_6h_IFNa.wig --step 50
perl bedgraph_to_wig.pl --bedgraph peak/STAT1_6h_IFNa_control_lambda.bdg --wig INP_6h_IFNa.wig --step 50
perl bedgraph_to_wig.pl --bedgraph peak/STAT1_30m_IFNa_treat_pileup.bdg --wig STAT1_30m_IFNa.wig --step 50
perl bedgraph_to_wig.pl --bedgraph peak/STAT1_30m_IFNa_control_lambda.bdg --wig INP_30m_IFNa.wig --step 50

# Visualization in IGB

In [ ]:
# on local computer, transfer files 
# install IGB,https://www.bioviz.org/otherDownloads.html 
mkdir CHIPseq_workshop
cd CHIPseq_workshop
scp zhuy16@helix.nih.gov:/spin1/users/classes/NIEHS_NGS/workshop/chip-seq/STAT1_6h_IFNa.wig .
scp zhuy16@helix.nih.gov:/spin1/users/classes/NIEHS_NGS/workshop/chip-seq/STAT1_30m_IFNa.wig .
scp zhuy16@helix.nih.gov:/spin1/users/classes/NIEHS_NGS/workshop/chip-seq/INP_6h_IFNa.wig .
scp zhuy16@helix.nih.gov:/spin1/users/classes/NIEHS_NGS/workshop/chip-seq/INP_30m_IFNa.wig .
drag these files into the IGB window to visualize

Procedure
1. load IGB
2. Species --> Homo sapiens 
3. genome version --> H_sapiens_Feb_2009
4. Drag .bdg .bigwig .bed files to the window.
5. click 'load data'
6. at the bottom --> click 'Advanced Search'-->IFIT1
7. select tracks and adjust scales --> at the bottom --> click 'Graph'
8. compare .bdg, .bigwig, .narrowpeak, .bed file format and see how they differ.